In [1]:
## This is demo for kuka reaching a with mpc and diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022

import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from diff_pin_costs import DiffFK
from inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [2]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")

/opt/openrobots/lib/python3.8/site-packages/pinocchio/shortcuts.py:45: UserWarning: You passed package dir(s) via argument geometry_model and provided package_dirs.
  geom_model = pin.buildGeomFromUrdf(model, filename, geometry_type, package_dirs)


In [3]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [674]:
dfk = DiffFK.apply
def quadratic_loss(q_pred, x_des, nq, n_col):
    loss = 2*torch.linalg.norm(dfk(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 0.5*torch.linalg.norm(q_pred[-nq:])

    for i in range(n_col):    
        loss += 0.8*torch.linalg.norm(dfk(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 1e-3*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 1e-2*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+2)*nq]) # joint regularization
    
    return loss

In [675]:
nq = model.nq
nv = model.nv

n_col = 10
u_max = [2.5,2.5,2.5, 1.5, 1.5, 1.5, 1.0]

lr = 1e-1
eps = 100

# q_des = np.hstack(((np.pi/4)*(np.random.randint(0, 2, size = 5)), np.zeros(2)))

# dq_des = np.zeros_like(q_des)
# pin.forwardKinematics(model, data, q_des, dq_des, np.zeros(nv))
# pin.updateFramePlacements(model, data)

x_des_arr = torch.tensor([[0.5, -0.4, 0.4]]) #[0.6, 0.4, 1.0]

x_init = np.zeros(2*nq)
x_init[0:nq] = np.hstack(((np.pi/12)*(np.random.randint(3, 6, size = 5)), np.zeros(2)))
x_init[nq:] = 0.2*2*(np.random.rand(len(q_des)) - 0.5)



In [677]:
x_in = x_init
x_des = x_des_arr[np.random.randint(len(x_des_arr))]

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des.detach().numpy()))

q_init = np.hstack(((np.pi/8)*np.random.choice(list(range(-2, -1)) + list(range(1, 3)), size=(4)), np.zeros(3)))

viz.display(q)

for j in range(15):

    ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0, isvec=False)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)
    
    
    i = 0
    loss = 1000.
    old_loss = 10000.
    
    while loss > 0.03 and i < eps and abs(old_loss - loss) > 5e-5:
        x_pred = ioc(x_in) 

        old_loss = loss

        loss = quadratic_loss(x_pred, x_des, nq, n_col)
        print("Index :" + str(i) + " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    x_pred = ioc(x_in).detach().numpy()

    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]

        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.05)
    
    x_in = torch.tensor(x_pred[-2*nq:])
    print(x_in[-nq:])

tensor([-0.2418, -0.0763,  0.0975,  0.2486, -0.0083, -0.3355,  0.1159],
       dtype=torch.float64)
tensor([-0.3900,  0.0617,  0.3475,  0.6986, -0.0251, -0.0355,  0.0159],
       dtype=torch.float64)
tensor([-0.1400,  0.0617,  0.0975, -0.0514, -0.0299, -0.0355,  0.0159],
       dtype=torch.float64)
tensor([-0.0376,  0.0068,  0.0975, -0.0514, -0.0299, -0.0355,  0.0159],
       dtype=torch.float64)
tensor([ 0.0097, -0.0053, -0.0017, -0.2014, -0.0067, -0.0034,  0.0159],
       dtype=torch.float64)
tensor([ 0.0211, -0.0053, -0.0017, -0.0514, -0.0067, -0.0034,  0.0159],
       dtype=torch.float64)
tensor([ 0.0153,  0.0187, -0.0017,  0.0986, -0.0067, -0.0034,  0.0012],
       dtype=torch.float64)
tensor([ 0.0153,  0.0042,  0.0017, -0.0514, -0.0067, -0.0034,  0.0012],
       dtype=torch.float64)
tensor([ 0.0153,  0.0042,  0.0017, -0.0006, -0.0067, -0.0034,  0.0012],
       dtype=torch.float64)
tensor([ 0.0153,  0.0320, -0.0267, -0.0006, -0.0067, -0.0034,  0.0012],
       dtype=torch.float64)


In [548]:
torch.utils.cmake_prefix_path

'/usr/local/lib/python3.8/dist-packages/torch/share/cmake'